# CatBoost Hyperparameter Optimization

Optimize CatBoost hyperparameters to improve CV score.

Parameters to test:
- Learning rates: [0.01, 0.03, 0.05, 0.1] with proportional iterations
- Depths: [4, 5, 6, 7, 8]
- Regularization: reg_lambda values [1, 3, 5, 10]
- Early stopping rounds: [30, 50, 100]

In [ ]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
import warnings
warnings.filterwarnings('ignore')

SEED = 42
np.random.seed(SEED)

print("Loading data...")
train_df = pd.read_csv('/home/code/data/train.csv')
test_df = pd.read_csv('/home/code/data/test.csv')

print(f"Train: {train_df.shape}, Test: {test_df.shape}")

In [ ]:
def create_features(df):
    """Create engineered features for the model"""
    df_new = df.copy()
    
    # Original numerical features
    num_features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
    
    # Log1p transformations
    for col in num_features:
        df_new[f'{col}_log1p'] = np.log1p(df_new[col])
    
    # Product features (from winning solutions)
    df_new['Weight_Duration'] = df_new['Weight'] * df_new['Duration']
    df_new['Duration_Heart_Rate'] = df_new['Duration'] * df_new['Heart_Rate']
    df_new['Height_Weight'] = df_new['Height'] * df_new['Weight']
    
    # Ratio features
    df_new['Weight_Height'] = df_new['Weight'] / (df_new['Height'] + 1e-6)
    
    # BMI feature (Body Mass Index approximation)
    df_new['BMI'] = df_new['Weight'] / ((df_new['Height'] / 100) ** 2 + 1e-6)
    
    return df_new

# Create features
train_feat = create_features(train_df)
test_feat = create_features(test_df)

# Define feature columns
feature_cols = [col for col in train_feat.columns if col not in ['id', 'Calories']]
cat_features = ['Sex'] if 'Sex' in feature_cols else []

print(f"Features: {len(feature_cols)}")
print(f"Categorical: {cat_features}")

In [ ]:
# Prepare data
X = train_feat[feature_cols]
y = train_feat['Calories']
X_test = test_feat[feature_cols]

print(f"X shape: {X.shape}, y shape: {y.shape}, X_test shape: {X_test.shape}")

In [ ]:
def train_catboost(params, X_train, y_train, X_val, y_val, cat_features):
    """Train CatBoost model and return validation score"""
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)
    
    model = CatBoostRegressor(**params)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)
    
    pred_val = model.predict(val_pool)
    score = np.sqrt(mean_squared_log_error(y_val, np.clip(pred_val, 0, None)))
    
    return score, model.best_iteration_

# Test configurations
configs = [
    # Learning rate sweep (adjust iterations proportionally)
    {'name': 'lr_0.01', 'learning_rate': 0.01, 'iterations': 2500, 'depth': 6},
    {'name': 'lr_0.03', 'learning_rate': 0.03, 'iterations': 1500, 'depth': 6},
    {'name': 'lr_0.05', 'learning_rate': 0.05, 'iterations': 1000, 'depth': 6},  # baseline
    {'name': 'lr_0.1', 'learning_rate': 0.1, 'iterations': 500, 'depth': 6},
    
    # Depth exploration
    {'name': 'depth_4', 'learning_rate': 0.05, 'iterations': 1000, 'depth': 4},
    {'name': 'depth_5', 'learning_rate': 0.05, 'iterations': 1000, 'depth': 5},
    {'name': 'depth_7', 'learning_rate': 0.05, 'iterations': 1000, 'depth': 7},
    {'name': 'depth_8', 'learning_rate': 0.05, 'iterations': 1000, 'depth': 8},
]

base_params = {
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'random_seed': SEED,
    'verbose': False,
    'allow_writing_files': False,
    'task_type': 'CPU'
}

print("Testing configurations...")
print("="*60)

results = []
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

for config in configs:
    print(f"\nTesting: {config['name']}")
    print(f"  lr={config['learning_rate']}, depth={config['depth']}, iter={config['iterations']}")
    
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
        
        params = {**base_params, **config}
        score, best_iter = train_catboost(params, X_tr, y_tr, X_va, y_va, cat_features)
        fold_scores.append(score)
        
        print(f"  Fold {fold}: {score:.6f} (best_iter: {best_iter})")
    
    mean_score = np.mean(fold_scores)
    std_score = np.std(fold_scores)
    
    results.append({
        'name': config['name'],
        'learning_rate': config['learning_rate'],
        'depth': config['depth'],
        'iterations': config['iterations'],
        'mean_score': mean_score,
        'std_score': std_score,
        'fold_scores': fold_scores
    })
    
    print(f"  Mean: {mean_score:.6f} ± {std_score:.6f}")
    print(f"  Folds: {fold_scores}")

In [ ]:
# Display results summary
print("\n" + "="*80)
print("HYPERPARAMETER OPTIMIZATION RESULTS")
print("="*80)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('mean_score')

for _, row in results_df.iterrows():
    print(f"{row['name']:15s} | lr={row['learning_rate']:.2f} | depth={row['depth']} | "
          f"score={row['mean_score']:.6f} ± {row['std_score']:.6f}")

print("\n" + "="*80)
print("Best configuration:")
best = results_df.iloc[0]
print(f"{best['name']} - Score: {best['mean_score']:.6f} ± {best['std_score']:.6f}")
print(f"Parameters: lr={best['learning_rate']}, depth={best['depth']}, iter={best['iterations']}")

# Compare to baseline
baseline_score = 0.202383
print(f"\nBaseline score: {baseline_score:.6f}")
print(f"Improvement: {best['mean_score'] - baseline_score:+.6f}")

In [ ]:
# Train final model with best parameters and generate predictions
best_config = {
    'learning_rate': best['learning_rate'],
    'depth': best['depth'],
    'iterations': best['iterations'],
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'random_seed': SEED,
    'verbose': False,
    'allow_writing_files': False,
    'task_type': 'CPU'
}

print(f"Training final model with best parameters: {best['name']}")
print(f"Parameters: {best_config}")

oof_predictions = np.zeros(len(train_df))
test_predictions = np.zeros(len(test_df))
fold_scores = []

fold = 1
for train_idx, val_idx in kf.split(X):
    print(f"\nFold {fold}/5")
    
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]
    
    train_pool = Pool(X_tr, y_tr, cat_features=cat_features)
    val_pool = Pool(X_va, y_va, cat_features=cat_features)
    
    model = CatBoostRegressor(**best_config)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=False)
    
    # Validation predictions
    pred_va = model.predict(val_pool)
    oof_predictions[val_idx] = pred_va
    
    # Test predictions
    test_pool = Pool(X_test, cat_features=cat_features)
    pred_test = model.predict(test_pool)
    test_predictions += pred_test / n_folds
    
    # Score
    score = np.sqrt(mean_squared_log_error(y_va, np.clip(pred_va, 0, None)))
    fold_scores.append(score)
    
    print(f"  Fold {fold} RMSLE: {score:.6f} (best_iter: {model.best_iteration_})")
    
    fold += 1

final_score = np.mean(fold_scores)
final_std = np.std(fold_scores)

print("\n" + "="*60)
print("FINAL MODEL RESULTS")
print("="*60)
print(f"CV RMSLE: {final_score:.6f} ± {final_std:.6f}")
print(f"Individual folds: {fold_scores}")
print(f"Improvement over baseline: {final_score - baseline_score:+.6f}")

In [ ]:
# Create submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'Calories': np.clip(test_predictions, train_df['Calories'].min(), train_df['Calories'].max())
})

submission_path = '/home/submission/submission_004_catboost_hyperopt.csv'
submission.to_csv(submission_path, index=False)

print(f"\nSubmission saved: {submission_path}")

# Save OOF predictions
oof_df = pd.DataFrame({
    'id': train_df['id'],
    'oof_prediction': oof_predictions
})
oof_path = '/home/code/experiments/oof_004_catboost_hyperopt.csv'
oof_df.to_csv(oof_path, index=False)

print(f"OOF predictions saved: {oof_path}")